In [155]:
print("Assumes one model to one sample and one sample to one passage")
print("also assumes both sample platform and Omic will have model ID's on every row")
providerAbbr = "PMLB"
os.chdir("/home/afollette/jupyter_notebooks/populateOmicFromSamplePlatform/")

Assumes one model to one sample and one sample to one passage
also assumes both sample platform and Omic will have model ID's on every row


In [156]:
%%bash -s "$providerAbbr"
mkdir workingFiles 
rsync -avz  /home/afollette/Finder_Data_Repositories/pdxfinder-data/data/UPDOG/"$1"/* ./workingFiles

sending incremental file list
cna/
cna/PMLB_cna.tsv
mut/
mut/PMLB_mut.tsv

sent 785,375 bytes  received 66 bytes  1,570,882.00 bytes/sec
total size is 31,935,310  speedup is 40.66


mkdir: cannot create directory ‘workingFiles’: File exists


In [157]:
import pandas as ps
import os
import glob


In [158]:
def useSamplePlatformToPopulateOmic(omicDf,omicType):
    print("Running dataframe for datatype {}".format(omicType))
    columnsList = omicDf.columns.tolist()
    populatedOmic = ps.DataFrame()
    tmpDf = ps.DataFrame()
    for index, row in omicDf.iterrows():
        spRows = spDf[(spDf['sample_id'] == str(row['sample_id'])) & (spDf['molecular_characterisation_type'] == omicType)]           
        if len(spRows) == 1 :
            spRow = spRows.iloc[0]
            row['model_id']  = spRow['model_id']
            row['sample_origin'] = spRow['sample_origin']
            row['host_strain_nomenclature'] = spRow['host_strain_nomenclature']
            row['passage'] = spRow['passage']
            row['platform'] = spRow['platform']
            tmpDf = populatedOmic.append(row)
            populatedOmic = tmpDf
        else :
            print("No match for sample id {}".format(row['sample_id']))
    newHeaders = populatedOmic[columnsList]
    newFileName = "PMLB_pop_{0}.tsv".format(omicType)
    newHeaders.to_csv(newFileName, sep='\t', index=False)

In [159]:
def samplePlatformIsAmbiguous(spDf):
    isAmbiguous = True
    duplicatedSampleId = spDf.loc[ : , ['sample_id', 'molecular_characterisation_type']].duplicated()
    duplicatedModels = spDf[duplicatedSampleId].loc[:, ['model_id']].duplicated()
    if len(spDf[duplicatedSampleId].values) != 0:
        print("ambiguous sample id's detected. Aborting process")
    elif len(duplicatedModels.values) != 0:
        print("amibguious models detected. Aborting process")
    else:
        isAmbiguous = False
    return isAmbiguous

In [160]:
try:
    os.chdir('./workingFiles')
except:
    print('Already in ./workingFiles')

In [161]:
mutData = glob.glob("./mut/*_mut.tsv")[0]
cnaData = glob.glob("./cna/*_cna.tsv")[0]
spData = glob.glob("./*_sampleplatform*.tsv")[0]
spDf = ps.read_csv(spData, sep='\t', dtype=str)
spMut = spDf[spDf.molecular_characterisation_type == "mutation"]
spCNA = spDf[spDf.molecular_characterisation_type == "copy number alteration"]
mutDf = ps.read_csv(mutData,sep='\t', dtype=str)
cnaDf = ps.read_csv(cnaData, sep='\t', dtype=str)

In [162]:
if not samplePlatformIsAmbiguous(spDf):
    print("Passed samplePlatform checks. Starting to populate data")
    (spMut
            .merge(mutDf, on='sample_id', validate='one_to_many', how='inner')
            .reindex(columns=['model_id_x', 'sample_id', 'sample_origin_x', 'host_strain_nomenclature_x','passage_x', "symbol","biotype","coding_sequence_change","variant_class","codon_change","amino_acid_change","consequence","functional_prediction","read_depth","allele_frequency","chromosome","seq_start_position","ref_allele","alt_allele","ucsc_gene_id","ncbi_gene_id","ncbi_transcript_id","ensembl_gene_id","ensembl_transcript_id","variation_id","genome_assembly","platform"])
            .to_csv("./mut/" + providerAbbr + "_mut.tsv", index=False, sep='\t')        
               )
    
    testFrame =(spMut
            .merge(mutDf, on='sample_id', validate='one_to_many', how='left')
            .query('model_id_y != model_id_y')
             .to_csv("./mut/Left_droppedPoints.tsv",index=False, sep='\t')
               )
    
    (spMut
            .merge(mutDf, on='sample_id', validate='one_to_many', how='right')
            .query('model_id_x != model_id_x')
            .to_csv("./mut/right_droppedPoints.tsv",index=False, sep='\t')
     )
    
    
    (spCNA
            .merge(mutDf, on='sample_id', validate='one_to_many', how='inner')
            .reindex(columns=['model_id_x', 'sample_id', 'sample_origin_x', 'passage_x', 'host_strain_nomenclature_x', "model_id","sample_id","sample_origin","passage","host_strain_nomenclature","chromosome","seq_start_position","seq_end_position","symbol","ucsc_gene_id","ncbi_gene_id","ensembl_gene_id","log10r_cna","log2r_cna","fold_change","copy_number_status","gistic_value","picnic_value","genome_assembly","platform"])
            .to_csv("./cna/" + providerAbbr + "_cna.tsv", index=False, sep='\t')        
               )
    (spCNA
            .merge(mutDf, on='sample_id', validate='one_to_many', how='left')
            .query('model_id_y != model_id_y')
            .to_csv("./cna/left_droppedPoints.tsv",index=False, sep='\t')
     )
    
    (spCNA
            .merge(mutDf, on='sample_id', validate='one_to_many', how='right')
            .query('model_id_x != model_id_x')
            .to_csv("./cna/right_droppedPoints.tsv",index=False, sep='\t')
     )

Passed samplePlatform checks. Starting to populate data


In [153]:
testFrame.query('model_id_y != model_id_y')

,Field,sample_id,sample_origin_x,passage_x,engrafted_tumor_collection_site,model_id_x,host_strain_name,host_strain_nomenclature_x,molecular_characterisation_type,platform_x,...,ref_allele,alt_allele,ucsc_gene_id,ncbi_gene_id,ncbi_transcript_id,ensembl_gene_id,ensembl_transcript_id,variation_id,genome_assembly,platform_y
0,Example,CRC0228PRH0000000000D01000,xenograft,0,mammary glad,CRC0228PR,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mutation,Affymetrix SNP6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,64842,patient,non applicable,non applicable,PMLB64842,NaN,NaN,mutation,Targeted Next generation sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,NaN,73123,patient,non applicable,non applicable,PMLB73123,NaN,NaN,mutation,Targeted Next generation sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,NaN,2393,xenograft,3,mammary fat pad,PMLB2393,NOD SCID,NOD.CB17-Prkdcscid/J,mutation,Targeted Next generation sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,2435,xenograft,3,mammary fat pad,PMLB2435,NOD SCID,NOD.CB17-Prkdcscid/J,mutation,Targeted Next generation sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4277,NaN,POP_176_Li_X,xenograft,2,flank,PMLB_POP176,NOD SCID,NOD.CB17-Prkdcscid/J,mutation,whole exome sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4278,NaN,POP_177_Li_X,xenograft,1,flank,PMLB_POP177,NOD SCID,NOD.CB17-Prkdcscid/J,mutation,whole exome sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4279,NaN,POP_179_Li_X,xenograft,1,flank,PMLB_POP179,NOD SCID,NOD.CB17-Prkdcscid/J,mutation,whole exome sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4280,NaN,POP_184_Li_X,xenograft,unknown,flank,PMLB_POP184,NOD SCID,NOD.CB17-Prkdcscid/J,mutation,whole exome sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
